In [2]:
import os
import random
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import tqdm
from scipy.io import loadmat
import pickle

In [3]:
unprocessed_data = loadmat("data/dtseries1200.mat")["ata"]

use_data = np.delete(unprocessed_data, slice(29696+29716,91282),0) 

Left_data = np.delete(use_data, slice(29696,29696+29716),0) 

Right_data = np.delete(use_data, slice(0,29696),0) 

In [4]:
with open('data/atlas_to_coordinate.pickle', 'rb') as f:
    
    saved_atlas_to_coordinate = pickle.load(f)

In [5]:
# make a dict mapping euclidean coord to atlas coord

Euclidean_to_atlas = {}
for j in range(130):
    for i in range(66):
        
        if j == 0 or i == 65:
            
            Euclidean_to_atlas[(i,j)] = "zero"
        else:
            Euclidean_to_atlas[(i,j)] = (0, j-1, 64-i)

for j in range(130):
    for i in range(66,132):
        
        if j == 0 or i == 131:
            Euclidean_to_atlas[(i,j)] = "zero"
            
        else:
            Euclidean_to_atlas[(i,j)] = (1, j-1, 130-i)
        
for j in range(130):
    for i in range(132, 198):
        
        if j == 0 or i == 197:
            Euclidean_to_atlas[(i,j)] = "zero"
            
        else:
            Euclidean_to_atlas[(i,j)] = (2, j-1, 196-i)
        
        
for j in range(130):
    for i in range(198, 264):
        
        if j == 0 or i == 263:
            Euclidean_to_atlas[(i,j)] = "zero"
            
        else:
            Euclidean_to_atlas[(i,j)] = (3, j-1, 262-i)
        
        
for j in range(130):
    for i in range(264, 330):
        
        if j == 0 or i == 329:
            Euclidean_to_atlas[(i,j)] = "zero"
            
        else:
            Euclidean_to_atlas[(i,j)] = (4, j-1, 328-i)

In [6]:
# Prepare for G-Padding by making a dict 

where_to_get = {}

for i in range(1,65):
    
    where_to_get[(i,0)] = (67,i)

for i in range(67,131):
    
    where_to_get[(i,0)] = (133,i-66)
    
for i in range(133,197):
    
    where_to_get[(i,0)] = (199,i-132)
    
for i in range(199,263):
    
    where_to_get[(i,0)] = (265,i-198)
    
for i in range(265,329):
    
    where_to_get[(i,0)] = (1,i-264)
    
    
for j in range(1,65):
    
    where_to_get[(65,j)] = (67, j+64)
    
    where_to_get[(131,j)] = (133, j+64)
    
    where_to_get[(197,j)] = (199, j+64)
    
    where_to_get[(263,j)] = (265, j+64)
    
    where_to_get[(329,j)] = (1, j+64)
    
    
for j in range(65,129):
    
    where_to_get[(65,j)] = (j+2, 128)
    
    where_to_get[(131,j)] = (j+68, 128)
    
    where_to_get[(197,j)] = (j+134, 128)
    
    where_to_get[(263,j)] = (j+200, 128)
    
    where_to_get[(329,j)] = (j-64, 128)


def GPad(Tensor, where_to_get):
    
    for keys in where_to_get.keys():
        
        i, j = keys
        
        new_i, new_j = where_to_get[keys]
        
        for c in range(Tensor.size(0)):
        
            Tensor[c][i][j] = Tensor[c][new_i][new_j]

In [ ]:
# get data, interpolate, and save tensor that can be "conv2d"d


for data_num in tqdm.tqdm(range(1200)):
    
    L_data = Lz[:,data_num]
    R_data = Rz[:,data_num]
    
    numpy_return = np.zeros((2,330,130))
    
    for i in range(330):
        
        for j in range(130):
            
            if Euclidean_to_atlas[(i,j)] == "zero":
                
                numpy_return[0][i][j] = 0
                numpy_return[1][i][j] = 0
                
            else:
                
                x,y,z,Ldis0,Lnum0,Ldis1,Lnum1,Ldis2,Lnum2,Rdis0,Rnum0,Rdis1,Rnum1,Rdis2,Rnum2 = saved_atlas_to_coordinate[Euclidean_to_atlas[(i,j)]]  
                
                Ltotal = Ldis0 + Ldis1 + Ldis2
                
                L = [(Ldis0, Lnum0), (Ldis1, Lnum1), (Ldis2, Lnum2)]
                
                L.sort()
                
                numpy_return[0][i][j] = ( L[0][0]*L_data[int(L[2][1])] + L[1][0]*L_data[int(L[1][1])] + L[2][0]*L_data[int(L[0][1])] ) / Ltotal   
                
                
                Rtotal = Rdis0 + Rdis1 + Rdis2
                
                R = [(Rdis0, Rnum0), (Rdis1, Rnum1), (Rdis2, Rnum2)]
                
                R.sort()
                
                numpy_return[1][i][j] = ( R[0][0]*R_data[int(R[2][1])] + R[1][0]*R_data[int(R[1][1])] + R[2][0]*R_data[int(R[0][1])] ) / Rtotal   
                
                
    
    tensor_return = torch.from_numpy(numpy_return)
    
    GPad(tensor_return, where_to_get)
    
    tensor_true_return = torch.zeros((1,330,261))
    
    
    
    for i in range(330):
        
        for j in range(130):
            
            tensor_true_return[0][i][j] = tensor_return[0][i][j]
            
            tensor_true_return[0][i][131+j] = tensor_return[1][i][j]
    
    
    torch.save(tensor_true_return, "{}.pt".format(data_num))